In [1]:
from pprint import pprint

print('Hello, World!')

Hello, World!


In [2]:
from os import getenv

from lumibot.brokers import Ccxt

WOOFI_PRO_CONFIG = {
    "exchange_id": "woofipro",
    "api_key": getenv("WOOFIPRO_API_KEY"),
    "secret": getenv("WOOFIPRO_API_SECRET"),
    "accountId": getenv("WOOFIPRO_ACCOUNT_ID"),
    "privateKey": getenv("WALLET_PRIVATE_KEY"),
    "sandbox": True if getenv("WOOFIPRO_IS_SANDBOX", "true").lower() == "true" else False,
}

broker = Ccxt(WOOFI_PRO_CONFIG)
pprint(broker._fetch_balance())

Lumibot version 4.0.3
2025-08-28 00:51:19,044 | INFO | .env file loaded from: /home/balo/projects/hydra-strategy/.env
2025-08-28 00:51:19,048 | INFO | [unknown] Waiting for the socket stream connection to be established, 
                method _stream_established must be called
{'free': {}, 'info': {'holding': []}, 'total': {}, 'used': {}}


In [ ]:
from datetime import datetime
import requests

import ta
import pandas as pd

from lumibot.strategies import Strategy
from lumibot.traders import Trader
from lumibot.backtesting import PandasDataBacktesting
from lumibot.entities import Asset, Bars, Data

base_asset = 'ETH'
quote_asset = 'USDC'
resolution = '5'  # 5 minute

class HydraStrategy(Strategy):
    parameters = {
        'fast_ema_period': 50,
        'slow_ema_period': 20,
    }

    def initialize(self):
        self.set_market('24/7')

    def on_trading_iteration(self):
        base = Asset(base_asset, Asset.AssetType.CRYPTO)
        quote = Asset(quote_asset, Asset.AssetType.CRYPTO)

        bars: Bars = self.get_historical_prices(
            asset=base,
            quote=quote,
            timestep='5 minutes',
            length=100,
            timeshift=100 if self.first_iteration else 0
        )
        # print(bars.df)
        # raise Exception("Stopping after one iteration for demonstration purposes.")
        fast_ema = ta.trend.EMAIndicator(bars.df['close'], self.parameters["fast_ema_period"])
        slow_ema = ta.trend.EMAIndicator(bars.df['close'], self.parameters["slow_ema_period"])
        # print(f"Fast EMA: {fast_ema.ema_indicator()}")
        # raise Exception("Stopping after one iteration for demonstration purposes.")

        if fast_ema.ema_indicator().iloc[-1] > slow_ema.ema_indicator().iloc[-1]:
            if not self.get_position(base):
                self.create_order(
                    asset=base,
                    quote=quote,
                    quantity=10,
                    side='buy'
                )
        elif fast_ema.ema_indicator().iloc[-1] < slow_ema.ema_indicator().iloc[-1]:
            if self.get_position(base):
                self.create_order(
                    asset=base,
                    quote=quote,
                    quantity=10,
                    side='sell'
                )


strategy = HydraStrategy(broker=broker)

backtesting_start = datetime(2025, 8, 1)
backtesting_end = datetime(2025, 8, 27)

url = "https://api.orderly.org/v1/tv/history"
querystring = {
    "symbol": f"PERP_{base_asset}_{quote_asset}",
    "resolution": resolution,
    "from": str(int(backtesting_start.timestamp())),
    "to": str(int(backtesting_end.timestamp())),
}
response = requests.get(url, params=querystring)
data = response.json()
df = pd.DataFrame({
    'datetime': data.get('t', []),
    'open': data.get('o', []),
    'high': data.get('h', []),
    'low': data.get('l', []),
    'close': data.get('c', []),
    'volume': data.get('v', [])
})
df['datetime'] = pd.to_datetime(df['datetime'], unit='s')
df.set_index('datetime', inplace=True)
asset = Asset(base_asset, Asset.AssetType.CRYPTO)
quote = Asset(quote_asset, Asset.AssetType.CRYPTO)
pandas_data = {
    asset: Data(
        asset,
        df,
        timestep='minute',
        quote=quote,
    )
}
# print(df)

strategy.backtest(
    PandasDataBacktesting,
    backtesting_start=pandas_data[asset].datetime_start,
    backtesting_end=pandas_data[asset].datetime_end,
    budget=2000,
    pandas_data=pandas_data,
    benchmark_asset=asset,
    quote_asset=quote,
)

2025-08-28 03:39:26,989 | ERROR | _strategy.py:update_broker_balances:641 | [HydraStrategy] Error getting broker balances: woofipro not implemented yet.
Progress || 100.00%  [Elapsed: 0:03:01 ETA: 0:00:00] Portfolio Val: 2,000,000.00

/snap/core20/current/lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.38' not found (required by /lib64/libduktape.so.207)
Failed to load module: /home/balo/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Opening in existing browser session.


{'cagr': np.float64(0.0),
 'volatility': np.float64(0.0),
 'sharpe': 0,
 'max_drawdown': {'drawdown': np.float64(0.0),
  'date': Timestamp('2025-08-21 03:51:00-0400', tz='America/New_York')},
 'romad': 0,
 'total_return': np.float64(0.0)}

/snap/core20/current/lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.38' not found (required by /lib64/libduktape.so.207)
Failed to load module: /home/balo/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Opening in existing browser session.


In [6]:
print(datetime(2025, 8, 26).timestamp())

1756162800.0
